# REQUEST FILE GENERATION

In [3]:
import random
import networkx as nx
import pickle

def generate_requests(graph, num_requests, demand_type):
    result = []
    node_request = len(graph.nodes) 
    
    for _ in range(num_requests):
        source = random.choice(list(graph.nodes))
        destination = random.choice(list(graph.nodes))
        while destination == source:
            destination = random.choice(list(graph.nodes))
        if demand_type == 'variable':
            demand = 12.5 * random.choice([1, 2, 3, 4, 5, 6])
        else:
            demand = demand
        
        new_req = {
            "source": source,
            "destination": destination,
            "demand": demand
        }
        
        result.append(new_req)
        
    return result

if __name__ == "__main__":
    file_path = input("Enter file name : ")
    
    try:
        # graph = nx.read_gpickle(file_path+".gpickle")
        with open(file_path + ".gpickle", "rb") as f:
            graph = pickle.load(f)
        num_requests = int(input("Enter the number of requests: "))
        demand_type = input("Enter demand type (40 or 80 for fixed, or variable): ").lower()
        if demand_type == '40':
            demand = 40
            
            
        elif demand_type == '80':
            demand = 80
        elif demand_type == 'variable':
            demand = "variable"
        else:
            print("Invalid demand type.")
            exit()
        requests = generate_requests(graph, num_requests, demand_type)
        for req in requests:
            print(req)
    
    except FileNotFoundError:
        print("The specified gpickle file was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


{'source': 16, 'destination': 19, 'demand': 12.5}
{'source': 0, 'destination': 4, 'demand': 62.5}
{'source': 15, 'destination': 7, 'demand': 25.0}
{'source': 22, 'destination': 13, 'demand': 50.0}
{'source': 4, 'destination': 6, 'demand': 37.5}
{'source': 18, 'destination': 22, 'demand': 50.0}
{'source': 10, 'destination': 20, 'demand': 50.0}
{'source': 3, 'destination': 7, 'demand': 12.5}
{'source': 15, 'destination': 2, 'demand': 75.0}
{'source': 8, 'destination': 18, 'demand': 37.5}
{'source': 13, 'destination': 3, 'demand': 75.0}
{'source': 14, 'destination': 13, 'demand': 75.0}
{'source': 21, 'destination': 18, 'demand': 25.0}
{'source': 9, 'destination': 10, 'demand': 50.0}
{'source': 4, 'destination': 1, 'demand': 62.5}
{'source': 2, 'destination': 14, 'demand': 25.0}
{'source': 15, 'destination': 18, 'demand': 12.5}
{'source': 14, 'destination': 8, 'demand': 50.0}
{'source': 17, 'destination': 9, 'demand': 25.0}
{'source': 4, 'destination': 8, 'demand': 25.0}
{'source': 9, 'des

# DATA GENERATION

In [4]:
import networkx as nx

def load_network_graph(filename):
    return nx.read_gpickle(filename)

def sparse_regeneration_algorithm(network_graph, requests, distance_threshold , Frequency_Slot_Capacity ):
    results = [] 
    total_regen_primary=0
    total_regen_backup=0
    Total_Frequency_Slot=0
    Guard_Frequency_Slot=2
    for request in requests:
        source = request['source']
        destination = request['destination']
        demand = request['demand']  
        
        
        try:   
            primary_path = nx.dijkstra_path(graph, source=source, target=destination, weight='weight')
            total_distance_primary = sum(graph[primary_path[i]][primary_path[i+1]]['weight'] for i in range(len(primary_path) - 1))
            
            graph_backup = graph.copy()
            for i in range(len(primary_path) - 1):
                graph_backup.remove_edge(primary_path[i], primary_path[i+1])
            try:
                backup_path = nx.dijkstra_path(graph_backup, source=source, target=destination, weight='weight')
                total_distance_backup = sum(graph_backup[backup_path[i]][backup_path[i+1]]['weight'] for i in range(len(backup_path) - 1))
            except nx.NetworkXNoPath:
                backup_path = None
                total_distance_backup = None
            
            regenerations_primary = 0
            regenerations_backup = 0
            current_distance_primary = 0
            current_distance_backup = 0
            regeneration_nodes_primary = []
            regeneration_nodes_backup = []
            path_segments_primary = []
            path_segments_backup = []
            segment_start_primary = source
            segment_start_backup = source
            
#             FOR PRIMARY PATH
            
            for i in range(1, len(primary_path)):
                link_distance = graph[primary_path[i-1]][primary_path[i]]['weight']
                current_distance_primary += link_distance 
                N = (demand//Frequency_Slot_Capacity)+Guard_Frequency_Slot
                Total_Frequency_Slot+=N
                
                
                if current_distance_primary > distance_threshold:
                    regenerations_primary += 1
                    total_regen_primary+=1
                    regeneration_nodes_primary.append(primary_path[i-1])
                    
                    if segment_start_primary != primary_path[i-1]:
                        path_segments_primary.append((segment_start_primary, primary_path[i-1]))
                    segment_start_primary = primary_path[i-1]
                        
                    current_distance_primary -= distance_threshold
            
            path_segments_primary.append((segment_start_primary, destination))
            
#             FOR BACKUP PATH

            if backup_path:
                for i in range(1, len(backup_path)):
                    link_distance = graph_backup[backup_path[i-1]][backup_path[i]]['weight']
                    current_distance_backup += link_distance
                    
                    if current_distance_backup > distance_threshold:
                        regenerations_backup += 1
                        regeneration_nodes_backup.append(backup_path[i-1])
                        
                        if segment_start_backup != backup_path[i-1]:
                            path_segments_backup.append((segment_start_backup, backup_path[i-1]))
                        segment_start_backup = backup_path[i-1]
                        
                        current_distance_backup -= distance_threshold
                
                path_segments_backup.append((segment_start_backup, destination))
            
            results.append({
                'source': source,
                'destination': destination,
                'demand':demand,
                
                'primary_path': primary_path,
                'total_distance_primary': total_distance_primary,
                'path_segments_primary': path_segments_primary,
                'regenerations_primary': regenerations_primary,
                'regeneration_nodes': regeneration_nodes_primary,
                
                'backup_path':backup_path,
                'total_distance_backup': total_distance_backup,
                'path_segments_backup': path_segments_backup,
                'regenerations_backup': regenerations_backup,
                'regeneration_nodes_backup': regeneration_nodes_backup,
                
                'Total_Regenerations_Primary' : total_regen_primary,
                'Total_Frequency_Slot' : Total_Frequency_Slot,
                'Spectrum_Allocated':N,
                
            })
        except nx.NetworkXNoPath:
            print(f"No path found between {source} and {destination}.")
            results.append({
                'source': source,
                'destination': destination,
                'demand':None,
                'primary_path': None,
                'total_distance_primary': None,
                'backup_path':None,
                'total_distance_backup': None,
                'path_segments_primary': None,
                'regenerations_primary': None,
                'regeneration_nodes': None,
                'Total_Regenerations_Primary' : None,
                'Total_Frequency_slot' : None,
                'Spectrum_Allocated':None,
            })
    return results

network_graph=graph

distance_threshold = 2000  
Frequency_Slot_Capacity = 25

results = sparse_regeneration_algorithm(network_graph, requests, distance_threshold , Frequency_Slot_Capacity)

for result in results:
    print("\nRequest from {} to {}".format(result['source'], result['destination']))
    print("  Demand:",result["demand"])
    print("************************************************************")
    print("  Path (primary):", result['primary_path'])
    print("  Total Distance (primary):", result['total_distance_primary'])
    print("  Regenerations Required (primary):", result['regenerations_primary'])
    print("  Regeneration_at_nodes (primary):",result["regeneration_nodes"])
    print("  Spectrum Allocated:",result["Spectrum_Allocated"])
    print("  Segment (primary):",result["path_segments_primary"] )
    
    print("\n  Path (backup):", result['backup_path'])
    print("  Total Distance (Backup):", result['total_distance_backup'])
    print("  Regenerations Required (backup):", result['regenerations_backup'])
    print("  Regeneration_at_nodes (backup):", result["regeneration_nodes_backup"])
    print("  Segment (backup):", result["path_segments_backup"])
    
    
print("\n\nTotal_Regenerations:" ,result["Total_Regenerations_Primary"])
print("Total_Frequency_Slot:" ,result["Total_Frequency_Slot"])


Request from 16 to 19
  Demand: 12.5
************************************************************
  Path (primary): [16, 21, 20, 19]
  Total Distance (primary): 1850
  Regenerations Required (primary): 0
  Regeneration_at_nodes (primary): []
  Spectrum Allocated: 2.0
  Segment (primary): [(16, 19)]

  Path (backup): [16, 15, 14, 19]
  Total Distance (Backup): 2900
  Regenerations Required (backup): 1
  Regeneration_at_nodes (backup): [14]
  Segment (backup): [(16, 14), (14, 19)]

Request from 0 to 4
  Demand: 62.5
************************************************************
  Path (primary): [0, 1, 2, 4]
  Total Distance (primary): 2700
  Regenerations Required (primary): 1
  Regeneration_at_nodes (primary): [2]
  Spectrum Allocated: 4.0
  Segment (primary): [(0, 2), (2, 4)]

  Path (backup): [0, 5, 6, 3, 4]
  Total Distance (Backup): 3650
  Regenerations Required (backup): 1
  Regeneration_at_nodes (backup): [6]
  Segment (backup): [(0, 6), (6, 4)]

Request from 15 to 7
  Demand: 25.

# SPECTRUM ALLOCATION

In [5]:
import numpy as np

total_segments=0
segment_names = []

for result in results:
    if result["path_segments_primary"]:
        for segment in result["path_segments_primary"]:
            total_segments += 1
            segment_names.append(f"segment_primary_{segment[0]}_{segment[1]}")

    if result["path_segments_backup"]:
        for segment in result["path_segments_backup"]:
            total_segments += 1
            segment_names.append(f"segment_backup_{segment[0]}_{segment[1]}")

array = np.zeros((total_segments, 100))

print("\nTotal Segments:", total_segments)
print("Segment Names:")
for i, name in enumerate(segment_names):
    print(f"Row {i}: {name}")

print("\n2D Array initialized to zeros:")
print(array)
print(array.shape)



Total Segments: 437
Segment Names:
Row 0: segment_primary_16_19
Row 1: segment_backup_16_14
Row 2: segment_backup_14_19
Row 3: segment_primary_0_2
Row 4: segment_primary_2_4
Row 5: segment_backup_0_6
Row 6: segment_backup_6_4
Row 7: segment_primary_15_12
Row 8: segment_primary_12_7
Row 9: segment_backup_15_10
Row 10: segment_backup_10_8
Row 11: segment_backup_8_7
Row 12: segment_primary_22_16
Row 13: segment_primary_16_13
Row 14: segment_backup_22_17
Row 15: segment_backup_17_13
Row 16: segment_primary_4_6
Row 17: segment_backup_4_6
Row 18: segment_primary_18_20
Row 19: segment_primary_20_22
Row 20: segment_backup_18_11
Row 21: segment_backup_11_16
Row 22: segment_backup_16_22
Row 23: segment_primary_10_15
Row 24: segment_primary_15_20
Row 25: segment_backup_10_15
Row 26: segment_backup_15_20
Row 27: segment_primary_3_7
Row 28: segment_backup_3_7
Row 29: segment_primary_15_8
Row 30: segment_primary_8_2
Row 31: segment_backup_15_10
Row 32: segment_backup_10_5
Row 33: segment_backup_5_2

# CREATING LINK DISJOINT GROUP OF PRIMARY AND BACKUP PATH

In [7]:
disjoint_groups = []
backup_disjoint_groups = []  # Groups for backup paths

def are_paths_link_disjoint(path1, path2, graph):
    for i in range(len(path1) - 1):
        edge1 = (path1[i], path1[i + 1])
        edge1_reverse = (path1[i + 1], path1[i])
        for j in range(len(path2) - 1):
            edge2 = (path2[j], path2[j + 1])
            edge2_reverse = (path2[j + 1], path2[j])
            if edge1 == edge2 or edge1 == edge2_reverse:
                return False
    return True

def is_path_already_in_groups(path, groups):
    for group in groups:
        for existing_path in group:
            if path == existing_path:
                return True
    return False

# Map to track primary to backup path relationships
primary_to_backup = {}

for result in results:
    primary_path = result['primary_path']
    backup_path = result.get('backup_path')  # Optional if not all results have a backup path

    if primary_path is not None:
        primary_to_backup[tuple(primary_path)] = backup_path  # Store backup path for each primary path

        if is_path_already_in_groups(primary_path, disjoint_groups):
            continue
        
        placed = False
        for group in disjoint_groups:
            is_disjoint = True
            for existing_path in group:
                if not are_paths_link_disjoint(primary_path, existing_path, network_graph):
                    is_disjoint = False
                    break
            if is_disjoint:
                group.append(primary_path)
                placed = True
                break
        if not placed:
            disjoint_groups.append([primary_path])

# Group backup paths based on primary path groups
for primary_group in disjoint_groups:
    backup_group = []
    for primary_path in primary_group:
        backup_path = primary_to_backup.get(tuple(primary_path))
        if backup_path:
            backup_group.append(backup_path)
    backup_disjoint_groups.append(backup_group)

# Display primary path groups
print("\nDisjoint Groups of Primary Paths:")
for idx, group in enumerate(disjoint_groups, 1):
    print(f"\nGroup {idx}:\n")
    for path in group:
        print(path)

# Display backup path groups
print("\nDisjoint Groups of Backup Paths:")
for idx, group in enumerate(backup_disjoint_groups, 1):
    print(f"\nBackup Group {idx}:\n")
    for path in group:
        print(path)



Disjoint Groups of Primary Paths:

Group 1:

[16, 21, 20, 19]
[0, 1, 2, 4]
[15, 11, 12, 9, 7]
[22, 16, 12, 13]
[4, 3, 6]
[8, 10, 18]
[17, 13, 9]
[21, 15, 14, 10]
[2, 3]
[9, 8]
[11, 8, 5]
[21, 22, 23]
[5, 10]
[8, 6]
[15, 20]

Group 2:

[18, 19, 20, 21, 22]
[10, 11, 15, 20]
[3, 4, 7]
[13, 9, 8, 6, 3]
[4, 2, 1]
[5, 10, 18]
[7, 9, 12, 11]
[14, 15, 16, 17]
[8, 5]
[10, 8]
[13, 12, 16]

Group 3:

[15, 11, 8, 6, 2]
[21, 20, 19, 18]
[9, 8, 10]
[9, 13, 17]
[10, 11, 12]
[22, 21, 15, 14, 10]
[18, 10]
[22, 16, 12, 9]

Group 4:

[14, 15, 11, 12, 13]
[15, 20, 19, 18]
[4, 3, 6, 8]
[20, 21, 16, 12, 9]
[8, 9, 13, 17, 23]
[14, 10, 5, 0]
[11, 10, 18]
[15, 21, 22]

Group 5:

[2, 6, 8, 11, 15, 14]
[22, 21, 20]
[20, 19, 18]
[8, 9, 13]
[11, 12]
[5, 10, 14, 19]
[7, 9, 12, 16, 21]

Group 6:

[14, 15, 11, 8]
[18, 10, 8, 6]
[9, 12, 16, 21, 20]
[8, 9, 7]
[17, 13, 12, 11]

Group 7:

[1, 5, 8, 11, 15]
[17, 16, 21, 20, 19]
[13, 9, 8, 6]
[14, 15, 20]

Group 8:

[16, 15, 11, 8]
[23, 22, 21, 20, 19]
[23, 17, 13, 9, 8]


# SPECTRUM ALLOCATION